# Option settings 

In [3]:
n_epochs=10000 #number of epochs of training
batch_size=3000 #size of the batches
lr=0.00005 #adam: learning rate
b1=0.5  #"adam: decay of first order momentum of gradient")
b2=0.999 #adam: decay of first order momentum of gradient")
n_cpu=8
latent_dim=100
img_size=100
channels=3
n_critic=5
clip_value=0.01
sample_interval=1000
img_shape = (channels, img_size, img_size)
crop_size = 400
print(img_shape)

(3, 100, 100)


# Static Code

In [4]:
from tensorboard_logger import configure, log_value
import tensorboard
import argparse
import os
import numpy as np
import math
import sys
import glob
import time
import random
#import utils import Logger

# torchvision stuff
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchvision import datasets


# torch stuff
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset







from PIL import Image


class ImageDataset(Dataset):
    def __init__(self, folder_path, transforms_=None):
        self.transform = transforms.Compose(transforms_)
        self.files = sorted(glob.glob('%s/*.*' % folder_path))

    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img = self.transform(img)
        return img

    def __len__(self):
        return len(self.files)
    
    
cuda = True if torch.cuda.is_available() else False
print(cuda)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity
    
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty


True


# Initialize Models and Load Settings

In [5]:
# Loss weight for gradient penalty
lambda_gp = 10


# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# If cuda, use cuda
if cuda:
    generator.cuda()
    discriminator.cuda()
    


In [6]:
lr=0.00005
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b2, b2))
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
print(lr)

5e-05


# Load Previous model

In [5]:
state_g = torch.load("/home/jgmeyer2/vangan/gans/models/g250px_jpegs_bw_5k.model")
state_d = torch.load("/home/jgmeyer2/vangan/gans/models/d250px_jpegs_bw_5k.model")


generator.load_state_dict(state_g['state_dict'])
optimizer_G.load_state_dict(state_g['optimizer'])

discriminator.load_state_dict(state_d['state_dict'])
optimizer_D.load_state_dict(state_d['optimizer'])




# INitialize logger

In [7]:
configure("runs/bw250n4",flush_secs=5)

# Training loop

# make training function

In [17]:
1000 % sample_interval == 0

True

In [18]:


def train(dataloader, batch_size, n_epochs):
    start = time.time()
    batches_done=0
    for epoch in range(n_epochs):
        batchtimes=[float()]
        for i, imgs in enumerate(dataloader):
            ## monitor time

            # Configure input
            real_imgs = Variable(imgs.type(Tensor))
            # ---------------------
            #  Train Discriminator
            # ---------------------
            optimizer_D.zero_grad()
            # Sample noise as generator input
            z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
            # Generate a batch of images
            fake_imgs = generator(z)
            # Real images
            real_validity = discriminator(real_imgs)
            # Fake images
            fake_validity = discriminator(fake_imgs)
            # Gradient penalty
            gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
            # Adversarial loss
            d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
            d_loss.backward()
            optimizer_D.step()
            optimizer_G.zero_grad()
            # Train the generator every n_critic steps
            if i % n_critic == 0:
                # -----------------
                #  Train Generator
                # -----------------
                # Generate a batch of images
                fake_imgs = generator(z)
                # Loss measures generator's ability to fool the discriminator
                # Train on fake images
                fake_validity = discriminator(fake_imgs)
                g_loss = -torch.mean(fake_validity)
                g_loss.backward()
                optimizer_G.step()
                #logger.log(d_loss, g_loss, epoch, batches_done, num_batches)
                if batches_done % sample_interval ==0:
                    log_value('g_loss', g_loss, batches_done)
                    log_value('d_loss', d_loss, batches_done)

                if batches_done % sample_interval/10 == 0:
                    save_image(fake_imgs.data[:25], "molpics250px_bw/%d_c.png" % batches_done, nrow=5, normalize=True)

                end = time.time()
                batchtime = (end - start)/5

                print(
                    "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [batch time: %f]"
                    % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item(), batchtime)
                )
                batchtimes.append(batchtime/batch_size)
                start = time.time()
                batches_done += n_critic
        print("average time per picture = " +str(np.mean(batchtimes)))
        print("minutes per 100,000 pictures = "+str((np.mean(batchtimes)*100000)/60))

# Input data 

In [19]:
folder_path = "./zinc100k250pxJPEGbw/"
n_epochs=10000
#lr=0.00005
batch_size = 1000
transforms_ = [ transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset(folder_path, transforms_=transforms_),
                        batch_size=batch_size, shuffle=True, num_workers=8)
train(dataloader, batch_size, n_epochs)

[Epoch 0/10000] [Batch 0/100] [D loss: -0.528707] [G loss: 93.487953] [batch time: 0.280665]
[Epoch 0/10000] [Batch 5/100] [D loss: -0.557090] [G loss: 95.561409] [batch time: 0.332230]
[Epoch 0/10000] [Batch 10/100] [D loss: -0.578491] [G loss: 97.548805] [batch time: 0.299882]
[Epoch 0/10000] [Batch 15/100] [D loss: -0.600343] [G loss: 99.441063] [batch time: 0.299715]
[Epoch 0/10000] [Batch 20/100] [D loss: -0.637169] [G loss: 101.222595] [batch time: 0.300425]
[Epoch 0/10000] [Batch 25/100] [D loss: -0.666200] [G loss: 102.886726] [batch time: 0.298040]
[Epoch 0/10000] [Batch 30/100] [D loss: -0.695588] [G loss: 104.391830] [batch time: 0.302085]
[Epoch 0/10000] [Batch 35/100] [D loss: -0.737312] [G loss: 105.759705] [batch time: 0.312939]
[Epoch 0/10000] [Batch 40/100] [D loss: -0.786624] [G loss: 106.958160] [batch time: 0.314534]
[Epoch 0/10000] [Batch 45/100] [D loss: -0.810632] [G loss: 107.972832] [batch time: 0.307759]
[Epoch 0/10000] [Batch 50/100] [D loss: -0.831813] [G lo

[Epoch 4/10000] [Batch 15/100] [D loss: -2.085838] [G loss: -16.068401] [batch time: 0.303836]
[Epoch 4/10000] [Batch 20/100] [D loss: -2.099266] [G loss: -11.315461] [batch time: 0.297755]
[Epoch 4/10000] [Batch 25/100] [D loss: -2.090943] [G loss: -6.376019] [batch time: 0.297753]
[Epoch 4/10000] [Batch 30/100] [D loss: -2.123437] [G loss: -1.224420] [batch time: 0.298257]
[Epoch 4/10000] [Batch 35/100] [D loss: -2.142596] [G loss: 4.105524] [batch time: 0.307288]
[Epoch 4/10000] [Batch 40/100] [D loss: -2.154840] [G loss: 9.558982] [batch time: 0.304958]
[Epoch 4/10000] [Batch 45/100] [D loss: -2.156598] [G loss: 15.159222] [batch time: 0.306192]
[Epoch 4/10000] [Batch 50/100] [D loss: -2.169441] [G loss: 20.887394] [batch time: 0.303443]
[Epoch 4/10000] [Batch 55/100] [D loss: -2.149685] [G loss: 26.689892] [batch time: 0.299946]
[Epoch 4/10000] [Batch 60/100] [D loss: -2.130310] [G loss: 32.580658] [batch time: 0.300012]
[Epoch 4/10000] [Batch 65/100] [D loss: -2.089174] [G loss: 

[Epoch 8/10000] [Batch 30/100] [D loss: -2.140680] [G loss: 30.845840] [batch time: 0.302016]
[Epoch 8/10000] [Batch 35/100] [D loss: -2.165114] [G loss: 33.614216] [batch time: 0.297431]
[Epoch 8/10000] [Batch 40/100] [D loss: -2.101748] [G loss: 36.393642] [batch time: 0.299213]
[Epoch 8/10000] [Batch 45/100] [D loss: -2.088038] [G loss: 39.178822] [batch time: 0.302028]
[Epoch 8/10000] [Batch 50/100] [D loss: -2.029208] [G loss: 41.946560] [batch time: 0.301709]
[Epoch 8/10000] [Batch 55/100] [D loss: -2.035224] [G loss: 44.692268] [batch time: 0.299870]
[Epoch 8/10000] [Batch 60/100] [D loss: -2.006218] [G loss: 47.371456] [batch time: 0.299206]
[Epoch 8/10000] [Batch 65/100] [D loss: -1.961121] [G loss: 49.976612] [batch time: 0.303987]
[Epoch 8/10000] [Batch 70/100] [D loss: -2.011693] [G loss: 52.484863] [batch time: 0.299207]
[Epoch 8/10000] [Batch 75/100] [D loss: -1.987291] [G loss: 54.891106] [batch time: 1.184704]
[Epoch 8/10000] [Batch 80/100] [D loss: -1.966574] [G loss: 

[Epoch 12/10000] [Batch 45/100] [D loss: -2.311558] [G loss: 63.428020] [batch time: 0.286437]
[Epoch 12/10000] [Batch 50/100] [D loss: -2.341001] [G loss: 60.809624] [batch time: 0.286048]
[Epoch 12/10000] [Batch 55/100] [D loss: -2.414757] [G loss: 58.108871] [batch time: 0.286156]
[Epoch 12/10000] [Batch 60/100] [D loss: -2.464910] [G loss: 55.369183] [batch time: 0.286089]
[Epoch 12/10000] [Batch 65/100] [D loss: -2.556430] [G loss: 52.599968] [batch time: 0.288883]
[Epoch 12/10000] [Batch 70/100] [D loss: -2.595417] [G loss: 49.828091] [batch time: 0.283326]
[Epoch 12/10000] [Batch 75/100] [D loss: -2.676613] [G loss: 47.077038] [batch time: 0.283432]
[Epoch 12/10000] [Batch 80/100] [D loss: -2.700459] [G loss: 44.369179] [batch time: 0.283725]
[Epoch 12/10000] [Batch 85/100] [D loss: -2.744353] [G loss: 41.730198] [batch time: 0.386351]
[Epoch 12/10000] [Batch 90/100] [D loss: -2.791865] [G loss: 39.186329] [batch time: 0.276317]
[Epoch 12/10000] [Batch 95/100] [D loss: -2.826793

[Epoch 16/10000] [Batch 55/100] [D loss: -3.242701] [G loss: 118.543137] [batch time: 0.287103]
[Epoch 16/10000] [Batch 60/100] [D loss: -3.241320] [G loss: 118.560822] [batch time: 0.288147]
[Epoch 16/10000] [Batch 65/100] [D loss: -3.228854] [G loss: 118.454910] [batch time: 0.284519]
[Epoch 16/10000] [Batch 70/100] [D loss: -3.231519] [G loss: 118.216026] [batch time: 0.288434]
[Epoch 16/10000] [Batch 75/100] [D loss: -3.243748] [G loss: 117.831772] [batch time: 0.287288]
[Epoch 16/10000] [Batch 80/100] [D loss: -3.279331] [G loss: 117.287773] [batch time: 0.286282]
[Epoch 16/10000] [Batch 85/100] [D loss: -3.284075] [G loss: 116.579842] [batch time: 0.284267]
[Epoch 16/10000] [Batch 90/100] [D loss: -3.313042] [G loss: 115.697975] [batch time: 0.277072]
[Epoch 16/10000] [Batch 95/100] [D loss: -3.274240] [G loss: 114.630203] [batch time: 0.277820]
average time per picture = 0.00028318681035723004
minutes per 100,000 pictures = 0.4719780172620501
[Epoch 17/10000] [Batch 0/100] [D lo

...

[Epoch 69/10000] [Batch 85/100] [D loss: 14.493217] [G loss: 202.385757] [batch time: 0.301250]
[Epoch 69/10000] [Batch 90/100] [D loss: 15.489141] [G loss: 213.679443] [batch time: 0.294616]
[Epoch 69/10000] [Batch 95/100] [D loss: 16.570866] [G loss: 224.260696] [batch time: 0.297386]
average time per picture = 0.0002988364242372058
minutes per 100,000 pictures = 0.4980607070620097
[Epoch 70/10000] [Batch 0/100] [D loss: 18.013340] [G loss: 233.892181] [batch time: 0.521174]
[Epoch 70/10000] [Batch 5/100] [D loss: 19.453272] [G loss: 242.702362] [batch time: 0.335705]
[Epoch 70/10000] [Batch 10/100] [D loss: 20.532253] [G loss: 251.070129] [batch time: 0.307800]
[Epoch 70/10000] [Batch 15/100] [D loss: 22.509598] [G loss: 258.099457] [batch time: 0.318462]
[Epoch 70/10000] [Batch 20/100] [D loss: 24.544920] [G loss: 264.206818] [batch time: 0.310118]
[Epoch 70/10000] [Batch 25/100] [D loss: 26.334068] [G loss: 269.726318] [batch time: 0.301193]
[Epoch 70/10000] [Batch 30/100] [D loss

Process Process-20344:
Process Process-20338:
Process Process-20341:
Process Process-20343:
Process Process-20342:
Process Process-20340:
Process Process-20337:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._po

  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 76, in __call__
    return F.to_tensor(pic)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])


KeyboardInterrupt: 

  File "<ipython-input-2-94d6cbab4dc5>", line 44, in __getitem__
    img = self.transform(img)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torchvision/transforms/functional.py", line 70, in to_tensor
    img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
KeyboardInterrupt
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/PIL/Image.py", line 722, in tobytes
    self.load()
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/PIL/ImageFile.py", line 231, in load
    n, err_code = decoder.decode(b)
KeyboardInterrupt


# Save models

In [20]:
os.makedirs("/home/jgmeyer2/vangan/gans/models",exist_ok=True)
PATH = "/home/jgmeyer2/vangan/gans/models/"
modelid="250px_jpegs_bw_lr0p00005_2469ep"
epoch=100

state_g = {
    'epoch': epoch,
    'state_dict': generator.state_dict(),
    'optimizer': optimizer_G.state_dict()
    }
torch.save(state_g, PATH+"g"+modelid+".model")

state_d = {
    'epoch': epoch,
    'state_dict': discriminator.state_dict(),
    'optimizer': optimizer_D.state_dict()
    }
torch.save(state_d, PATH+"d"+modelid+".model")
print("saved models @")
print(epoch)


#def save_model(net, optim, ckpt_fname):
#    state_dict = net.module.state_dict()
#    for key in state_dict.keys():
#        state_dict[key] = state_dict[key].cpu()
#        torch.save({
#            'epoch': epoch,                                                                                                                                                                                     
#            'state_dict': state_dict,                                                                                                                                                                                
#            'optimizer': optim},                                                                                                                                                                                     
#            ckpt_fname)

saved models @
100


# view training process

In [ ]:
#tensorboard
!python -m tensorboard.main --logdir runs

TensorBoard 1.10.0 at http://minilearn:6006 (Press CTRL+C to quit)
W0917 09:41:16.546699 Thread-1 application.py:276] path /[[_dataImageSrc]] not found, sending 404
W0917 09:41:16.546699 140297468602112 application.py:276] path /[[_dataImageSrc]] not found, sending 404
W0917 09:41:16.567598 Thread-1 application.py:276] path /[[_imageURL]] not found, sending 404
W0917 09:41:16.567598 140297468602112 application.py:276] path /[[_imageURL]] not found, sending 404
